In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
from scipy.stats import skew, zscore
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../')
from cleaners.cleaners import clean_dataframe, clean_dataframe_columns, unique_values_count, n_counts_and_values

In [ ]:
# define path

path = '../data/yellow'
all_files = glob.glob(path + "/*.parquet")

There are over 400M rows on the raw concatenated DF - I think this may be too taxing for models and a drag to import - reduced to 40K random sample to start with

In [ ]:
# uses function to load, clean column names, and sample each file

fraction = 0.0001  # sample fraction of rows
seed = 33 # use seed for consistency

# uses function to clean df columns
sampled_dfs = [clean_dataframe_columns(pd.read_parquet(file).sample(frac=fraction)) for file in all_files]

# concat all sampled names
final_df = pd.concat(sampled_dfs, ignore_index=True)

In [ ]:
# examine frame/columns
final_df.head()

In [ ]:
#looking at payment types as there are some differences from the reference materials

result = unique_values_count(final_df, 'payment_type')
print(result)

In [ ]:
final_df.shape

In [ ]:
#vendor ID counts should only be 1 or 2 - dropping this column in any case

vendorid_counts = final_df['vendorid'].value_counts()

print(vendorid_counts)

In [ ]:
# shorten syntax
df=final_df

In [ ]:
# view 100 random sampled rows

sampled_df = df.sample(n=100)
display(sampled_df)

In [ ]:
print(sampled_df.dtypes)

In [ ]:
# there were 2 airport fee columns, we need to stardardize before the concat - done at import

In [ ]:
# there are only 2 (the other instances appear to be typos) so this column can be dropped since such general info is not of use to us
# the improvement_surcharge is a standard add-on fall all fares so we can drop this too
# the airport fee is redundant since the ratecode includes the airport info - drop
# congestion surcharge is also not of use for us as well as store/fwrd flag, mta tax and extra (incorporated already in fare info) - drop.


final_df.drop(columns=['vendorid', 'improvement_surcharge', 'store_and_fwd_flag', 'mta_tax', 'extra', 'airport_fee', 'congestion_surcharge'], inplace=True)

In [ ]:
final_df.shape

In [ ]:
display(final_df)

In [ ]:
# looking at unique values for payment types using my function

result = unique_values_count(final_df, 'payment_type')
print(result)

In [ ]:
# uses my function to list values for each column (the user can speicfy how many columns) and the number of instances of each value
display(n_counts_and_values(final_df))

In [ ]:
# examining the rows with NaNs

pd.set_option('display.max_rows', 500)  # Set the maximum display rows to 500
nan_rows = final_df[final_df['passenger_count'].isna()]

nan_rows


The NaNs and 0s for passenger_count and the other 2 columns make up ~ 1.2% of the data and are randomly distributed so it is safe to drop these rows.
They could theoretically be fares without passengers but there is no info in the documentation about so I prefer not to guess in this instance without significant domain knowledge.

In [ ]:
# removing the rows with NaNs
final_df.dropna(subset=['passenger_count'], inplace=True)

In [ ]:
display(final_df)

In [ ]:
# I would like to add a new column to define a trip duration using the pickup/drop

In [ ]:
final_df.head()

In [ ]:
# creates new col -- trip_duration
final_df['trip_duration'] = (final_df['tpep_dropoff_datetime'] - final_df['tpep_pickup_datetime']).astype('timedelta64[us]')  
final_df.head()

In [ ]:
# convert the timedelta format to seconds (float)
final_df['trip_duration_seconds'] = final_df['trip_duration'].dt.total_seconds()


In [ ]:
print(final_df['trip_duration_seconds'].dtype)

In [ ]:
# drop previous column as it is now in secoonds
df.drop('trip_duration', axis=1, inplace=True)


In [ ]:
# check this col for NaNs or non-positive values 

In [ ]:
final_df.head()

In [ ]:
# last check for NaNs
num_rows_with_nan = final_df.isnull().any(axis=1).sum()
print(f"Number of rows with NaN values: {num_rows_with_nan}")


In [ ]:
final_df.shape

In [ ]:
# Save the cleaned and updated DataFrame as a CSV file (for tableau use) and a parquet file (retains datetime if opened again)
final_df.to_csv('../data/yellow/taxi_y_cleaned.csv', index=False)
final_df.to_parquet('../data/yellow/taxi_y_cleaned.parquet', index=False, engine='pyarrow')
